# **ResNet Image Classifier for EchoCardio-12**

This is the base image classifier for echonet, and is a resnet-50 with modified fully connected layer, relu and dropout. Trained from zerom as required for good performance.

### Install Weights and Biases + Login

In [1]:
!pip install wandb

import wandb

if wandb.run is not None:
  wandb.finish()

wandb.login()

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


wandb: Currently logged in as: frenchsatchel (lynerlabs). Use `wandb login --relogin` to force relogin


True

In [2]:
!pip install poetry lightning torchmetrics munch

Defaulting to user installation because normal site-packages is not writeable
DEPRECATION: flatbuffers 1.12.1-git20200711.33e2d80-dfsg1-0.6 has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of flatbuffers or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


### Clone and build dataset

* EchoCardio is too large to download as images, and ships as .mp4 files by default. The 'create_videoset.py' script will split it into frames organized so that it can be loaded as an image, video, or optical-flow dataset.

* The script accepts serval command line arguments for datapath, train/test split, output path, labels, seed, etc. See the code for details.

* The script will by default download the raw dataset, although you can change it via --url. Additionally it will **auto install and use ffmpeg and use that as the main backend, but will install open-cv if available or otherwise specified.**

In [5]:
!git clone https://github.com/satchelfrench/neo-echoset ./neo_echoset1

Cloning into './neo_echoset1'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 113 (delta 61), reused 85 (delta 34), pack-reused 0
Receiving objects: 100% (113/113), 53.56 KiB | 2.33 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Creating virtualenv neo-echoset-UjTpOypg-py3.10 in /home/ubuntu/.cache/pypoetry/virtualenvs
Installing dependencies from lock file

Package operations: 28 installs, 0 updates, 0 removals

  - Installing markupsafe (2.1.3)
  - Installing mpmath (1.3.0)
  - Installing certifi (2023.7.22)
  - Installing charset-normalizer (3.3.0)
  - Installing filelock (3.12.4)
  - Installing idna (3.4)
  - Installing jinja2 (3.1.2)
  - Installing networkx (3.1)
  - Installing numpy (1.26.0)
  - Installing six (1.16.0)
  - Installing sympy (1.12)
  - Installing typing-extensions (4.8.0)
  - Installing urllib3 (2.0.5)
  - Installing joblib (1.3.2)
  - Installing pillow (10

In [6]:
import cv2
import lightning as L

cv2.__version__

#!cd ./neo_echoset/ && poetry run python create_videoset.py --kfold 5 --ext --out_path "./neo_echoset_ext/"

L.seed_everything(7)



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

## Model Definition

* Nothing fancy, most of the tweaking here has focused around optimizer schedule and the fc layers.

* We're using Pytorch Lightning to quickly abstract and prototype the model architecture

* Similarly the torchmetrics package will handle our metrics + graphs.



In [7]:
import lightning.pytorch as pl
import torchvision.models as models
import torch
import torch.nn as nn
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall, MulticlassF1Score, MulticlassConfusionMatrix, MulticlassPrecisionRecallCurve
from torch.nn.functional import cross_entropy, log_softmax

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

class ResNet(pl.LightningModule):
  def __init__(self, train_set_length, params, K, class_list, fold_metrics, fold_pr, fold_confmat):
    super().__init__()
    self.classes = class_list
    self.params = params
    self.sample_size = train_set_length

    self.fold_metrics = fold_metrics
    self.fold_pr = fold_pr
    self.fold_confmat = fold_confmat

    if self.params.network == 'resnet18':
      self.resnet = models.resnet18(num_classes = K)
      fc0_in = 512
    elif self.params.network == 'resnet34':
      self.resnet = models.resnet34(num_classes = K)
      fc0_in = 512
    elif self.params.network == 'resnet50':
      self.resnet = models.resnet50(num_classes = K)
      fc0_in = 512*4

    if self.params.fc1_layer:
      self.fc0 = nn.Linear(fc0_in, self.params.fc0)
      self.fc1 = nn.Linear(self.params.fc0, K)
    else:
      self.fc0 = nn.Linear(fc0_in, K)

    self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,
                               bias=False)
    self.dropout = nn.Dropout(p=self.params.dropout)
    self.relu = nn.ReLU(inplace=True) if self.params.activation == 'relu' else nn.LeakyReLU(0.01, inplace=True)

    metrics = MetricCollection([
        MulticlassAccuracy(K), MulticlassPrecision(K), MulticlassRecall(K),
        MulticlassF1Score(K)
    ])

    self.confmat = MulticlassConfusionMatrix(K, normalize='true')
    self.pr_curve = MulticlassPrecisionRecallCurve(K)

    self.train_metrics = metrics.clone(prefix='train_')
    self.val_metrics = metrics.clone(prefix='val_')
    self.test_metrics = metrics.clone(prefix='test_')

    #self.save_hyperparameters() # what to do here? wandb doesnt like..


  def forward(self, x):
    x = self.resnet.conv1(x)
    x = self.resnet.bn1(x)
    x = self.relu(x)
    x = self.resnet.maxpool(x)
    x = self.dropout(x)

    x = self.resnet.layer1(x)
    x = self.resnet.layer2(x)
    x = self.resnet.layer3(x)
    x = self.resnet.layer4(x)

    x = self.resnet.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.fc0(x)

    if self.params.fc1_layer:
      x = self.relu(x)
      x = self.dropout(x)
      x = self.fc1(x)

    return log_softmax(x, dim=1)

  def configure_optimizers(self):
    optimizer = torch.optim.SGD(
        self.parameters(),
        lr=self.params.lr, #0.01
        momentum=self.params.momentum,
        weight_decay=5e-4, #-5
    )
    steps_per_epoch = self.sample_size // self.params.batch_size +1
    scheduler_dict = {
        "scheduler": torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=self.params.lr,
            epochs=self.trainer.max_epochs,
            steps_per_epoch=steps_per_epoch,
            pct_start=self.params.pct_start,
            final_div_factor=self.params.div_factor,
            base_momentum=self.params.momentum
        ),
        "interval": "step",
    }
    return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

  def training_step(self, batch, batch_idx):
    samples, labels = batch
    y = self.forward(samples)
    loss = cross_entropy(y, labels)
    metrics = self.train_metrics(y, labels)

    self.log_dict(metrics, on_epoch=True)
    self.log("train_loss", loss)

    return loss

  def validation_step(self, batch, batch_idx):
    samples, labels = batch
    preds = self.forward(samples)
    loss = cross_entropy(preds, labels)
    self.val_metrics.update(preds, labels)
    self.log_dict({"val_loss": loss}, on_epoch=True)

  def on_validation_epoch_end(self):
    val_metrics = self.val_metrics.compute()
    self.log_dict(val_metrics)
    self.val_metrics.reset()

  def test_step(self, batch, batch_idx):
    samples, labels = batch
    preds = self.forward(samples)
    self.test_metrics.update(preds, labels)
    self.confmat.update(preds, labels)
    self.pr_curve.update(preds, labels)

    self.fold_metrics.update(preds, labels)
    self.fold_confmat.update(preds, labels)
    self.fold_pr.update(preds, labels)

  # Save to external structure for K-Wide Compute.
  def on_test_end(self):
    test_metrics = self.test_metrics.compute()
    self.logger.log_metrics(test_metrics)

    # Log Test Table
    self.logger.log_table(key="Test Result Table",
                          columns=list(test_metrics.keys()),
                          data=[[v for _,v in test_metrics.items()]])

    confmat = self.confmat.compute()
    fig_conf, ax_conf = plt.subplots(figsize=(10,10))
    sns.heatmap(confmat.cpu(),
                cmap='crest',
                ax=ax_conf,
                annot=True,
                fmt=".2f",
                xticklabels=self.classes,
                yticklabels=self.classes)

    ax_conf.set(xlabel="Predictions", ylabel="Target")
    fig_conf.savefig('conf.png', bbox_inches="tight")

    pr = self.pr_curve.compute()
    y, x = pr[:2]

    fig_pr, ax_pr = plt.subplots(figsize=(10,10))

    for i, (x_, y_) in enumerate(zip(x, y)):
      auc = torch.trapezoid(y_, x_, axis=-1)*-1
      auc = auc.detach().cpu().numpy()
      label = self.classes[i] + ", AUC={:.3f}".format(auc)
      sns.lineplot(x=x_.detach().cpu(), y=y_.detach().cpu(), ax=ax_pr, legend='full', label=label)

    ax_pr.set(xlabel="Recall", ylabel="Precision")
    fig_pr.savefig('pr.png', bbox_inches="tight")
    self.logger.log_image(key="Graphs",images=["conf.png", "pr.png"])
    self.test_metrics.reset()








A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 1077, in launch_instance
    app.start()
  File "/usr/lib/python3/dist-packages/ipykernel/kernelapp.py", line 

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

### Reusable dataloading
* We'll abstract this so that it can be called succintly in a train loop, which is better for sweeps.
* Clean our code for later




In [ ]:
from neo_echoset.dataloaders.image_dataset import FrameDataset
from torchvision.transforms import transforms
import torch.utils.data as data

# Make this work with multiple dataset types..
def build_dataloaders(dataset_root, annotations_path, preprocess, params, shuffle, drop_last, pin_memory):
  dataset = FrameDataset(
    root_dir=dataset_root,
    annotations_file=annotations_path,
    transform=preprocess
  )

  dataloader = data.DataLoader(dataset, batch_size=params.batch_size,
                                 shuffle=shuffle, drop_last=drop_last, num_workers=8,
                                 pin_memory=pin_memory)

  return dataset, dataloader

In [ ]:
'''Function for testing dataset'''
# import matplotlib.pyplot as plt
# import numpy as np
# import random
# from neo_echoset.datility.utils import flow_to_rgb
# import cv2

# %matplotlib inline
# def dataset_peek(dataset, labels_map, figsize=(8,8), cols=3, rows=3):
#   figure = plt.figure(figsize=figsize)

#   for i in range(1, cols*rows+1):
#       k = random.randint(1, len(dataset))
#       image, label = dataset[k]

#       figure.add_subplot(rows, cols, i)
#       plt.title(
#           labels_map[int(label)]
#       )

#       plt.axis("off")
#       plt.imshow(np.moveaxis(np.asanyarray(image), 0, -1))

#   plt.show()

## Enable a hyperparam sweep

In [ ]:
# sweep_config = {
#     'method': 'random'
# }

# sweep_config['metric']  = {
#     'name': 'val_acc',
#     'goal': 'maximize'
# }

# sweep_config['parameters'] = {
#     'activation': { 'value': ['relu']},
#     'fc1_layer': { 'values': [True, False]},
#     'network': {'value': 'resnet50'},
#     'fc0': {' values': [128, 256, 512, 1024, 2048]},
#     'batch_size': {'values': [16, 32, 64]},
#     'dropout': {'values': [0.4, 0.5, 0.6]},
#     'lr': {'distribution': 'q_uniform', 'q': 0.01,'min': 0.05,'max': 0.15},
#     'momentum': {'min': 0.3, 'max': 0.95},
#     'pct_start': {'distribution': 'q_uniform','min': 0.1,'max': 0.7,'q': 0.1},
#     'div_factor': {'values': [1e2, 1e3, 1e4]},
#     'epochs': {'distribution': 'q_uniform','q': 1.0,'min': 20.0,'max': 70.0}
# }

# sweep_id = wandb.sweep(sweep_config, project="neo-echonet-img-sweep")

### Let's Train
* Finally we'll establish our hyper parameters, dataset paths and preprocess
* Then run everything..

In [ ]:
import os
from munch import Munch

params = {
    'activation':'relu',
    'fc1_layer': True,
    'network': 'resnet50',
    'fc0': 2048,
    'batch_size': 16,
    'dropout': 0.6,
    'lr': 0.08,
    'momentum': 0.75,
    'pct_start': 0.4,
    'div_factor': 1e4,
    'epochs': 50, #50
    'resnet_ckpt': None,
}

params = Munch(params) # for attribute style access, consistent with wandb sweeps

# PATHS
DATASET_ROOT = os.path.join('./neo_echoset/neo_echoset_ext/')
# TRAIN_LABELS = os.path.join(DATASET_ROOT, 'train_annotations.txt')
# VALID_LABELS = os.path.join(DATASET_ROOT, 'valid_annotations.txt')
CHECKPOINT_PATH = './checkpoints/'
K = 16


Get a mapping of our classes from integer to string, for labelling and what not.

In [ ]:
import pickle

# Load Class Labels
with open("./neo_echoset/classes.pkl", "rb") as f:
    label_map = pickle.load(f)

labels_map = {v: k for k, v in label_map.items()}
labels_list = [labels_map[i] for i in range(len(labels_map.items()))]

print(labels_map)
print(labels_list)

In [ ]:
# Run Train and Eval
from lightning.pytorch.callbacks import LearningRateMonitor, ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger
import torch.nn as nn

train_preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(),
        transforms.RandomRotation(25),
        transforms.Resize(232, antialias=True),  # image batch, resize smaller edge to 299
        transforms.CenterCrop(224),  # image batch, center crop to square 299x299
        transforms.RandomAutocontrast(p=0.5),
        # transforms.RandomVerticalFlip(p=0.25),
        # transforms.RandomHorizontalFlip(p=0.25),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.ToTensor(),
])

valid_preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(),
        transforms.Resize(232, antialias=True),
        transforms.CenterCrop(224),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        transforms.ToTensor(),
])

def run(config=None, project_name=None):

    wandb_logger = WandbLogger(project=project_name, log_model=True)
    wandb_logger.experiment.config.update(config)

    # log data about current k-fold split
    with open(os.path.join(DATASET_ROOT, config.valid_path), 'r') as f:
      val_paths = f.readlines()
      c = [ sample_path.split('/')[0] for sample_path in val_paths ]
      val_cases = set(c)

    wandb.log({"validation_cases": val_cases, "fold": config.fold})


    trainer = L.Trainer(
                  default_root_dir='./drive/MyDrive/ckpts/',
                  accelerator="auto",
                  max_epochs=config.epochs,
                  callbacks=[
                      ModelCheckpoint(
                          dirpath='./drive/MyDrive/neo-echonet-ckpts/',
                          save_top_k=1,
                          monitor="val_loss",
                          mode="min"
                      ),
                      LearningRateMonitor("epoch")
                  ],
                  logger=wandb_logger
              )

    train_set, train_loader = build_dataloaders(DATASET_ROOT,
                                                 os.path.join(DATASET_ROOT, config.train_path),
                                                 train_preprocess,
                                                 config,
                                                 shuffle=True,
                                                 drop_last=False,
                                                 pin_memory=True)

    valid_set, valid_loader = build_dataloaders(DATASET_ROOT,
                                                 os.path.join(DATASET_ROOT, config.valid_path),
                                                 valid_preprocess,
                                                 config,
                                                 shuffle=False,
                                                 drop_last=False,
                                                 pin_memory=False)
    # Block for quick export of val only

    artifact = wandb_logger.download_artifact(config.resnet_ckpt, artifact_type="model")
    ckpt = f"./artifacts/{config.resnet_ckpt.split('/')[-1]}/model.ckpt"
    
    model = ResNet.load_from_checkpoint(
                   ckpt, # model path
                   train_set_length=len(train_set),
                   params=config,
                   K=K,
                   class_list=labels_list,
                   fold_metrics=config.metrics,
                   fold_pr=config.pr,
                   fold_confmat=config.confmat)
    
    # model = ResNet(train_set_length=len(train_set),
    #                params=config,
    #                K=K,
    #                class_list=labels_list,
    #                fold_metrics=config.metrics,
    #                fold_pr=config.pr,
    #                fold_confmat=config.confmat)

    #wandb_logger.watch(model, log_freq=100) # track gradients

    # Train the model
    # trainer.fit(model, train_loader, valid_loader)
    #wandb_logger.experiment.unwatch(model)

    # Run Test
    # val_result = trainer.test(model, dataloaders=valid_loader, verbose=False, ckpt_path="best")
    val_result = trainer.test(model, dataloaders=valid_loader, verbose=False)
    print(val_result)


**If we're sweeping, we'll run this cell here, instead of the one below.**

In [ ]:
# wandb.agent(sweep_id, run, count=5)

### Training Block

In [ ]:
# Script has already been run, so need to read the json file and grab file names
# Then iterate through them and pass each to run via config?
# Then find a way to aggregate Fold-wide metrics

with open("./neo_echoset/folded_data.pkl", "rb") as f:
    data_folds = pickle.load(f)

# Create metrics to track across folds
fold_metrics = MetricCollection([
        MulticlassAccuracy(K), MulticlassPrecision(K), MulticlassRecall(K),
        MulticlassF1Score(K)
    ])

fold_confmat = MulticlassConfusionMatrix(K, normalize='true')
fold_pr = MulticlassPrecisionRecallCurve(K)

# 16 class checkpoints 
resnet_folded_ckpts = [
  'lynerlabs/resnet-50-echo/model-gt7b7rob:v0',
  'lynerlabs/resnet-50-echo/model-7e3kce75:v0',
  'lynerlabs/resnet-50-echo/model-enqp3sol:v0',
  'lynerlabs/resnet-50-echo/model-h27khter:v0',
  'lynerlabs/resnet-50-echo/model-swie5erh:v0',
]
for i, (train_path, valid_path) in enumerate(data_folds):
  params.fold = i
  params.train_path= train_path
  params.valid_path = valid_path
  params.metrics = fold_metrics
  params.confmat = fold_confmat
  params.pr = fold_pr  
  params.resnet_ckpt = resnet_folded_ckpts[i]


  print(f"Running fold #{i}..")
  run(config=params, project_name="resnet-50-echo")
  wandb.finish()

# Compute Fold Metrics

metrics = fold_metrics.compute()
confmat = fold_confmat.compute()

print(metrics)

with open("./metrics.txt", "w") as m:
    m.writelines(str(metrics))

fig_conf, ax_conf = plt.subplots(figsize=(10,10))
sns.heatmap(confmat.cpu(),
            cmap='crest',
            ax=ax_conf,
            annot=True,
            fmt=".2f",
            xticklabels=labels_list,
            yticklabels=labels_list)

ax_conf.set(xlabel="Predictions", ylabel="Target")
fig_conf.savefig('kconf.png', bbox_inches="tight")

pr = fold_pr.compute()
y, x = pr[:2]

fig_pr, ax_pr = plt.subplots(figsize=(10,10))

for i, (x_, y_) in enumerate(zip(x, y)):
  auc = torch.trapezoid(y_, x_, axis=-1)*-1
  auc = auc.detach().cpu().numpy()
  label = labels_list[i] + ", AUC={:.3f}".format(auc)
  sns.lineplot(x=x_.detach().cpu(), y=y_.detach().cpu(), ax=ax_pr, legend='full', label=label)

ax_pr.set(xlabel="Recall", ylabel="Precision")
fig_pr.savefig('kpr.png', bbox_inches="tight")


### Le Fin

In [ ]:
wandb.finish()